In [ ]:
# For macOS (Apple Silicon) or Windows
!pip3 install torch torchvision torchaudio torch_geometric

In [3]:
from torch_geometric.datasets.qm9 import QM9

data = QM9(root='/tmp/QM9')
print(f"Data length: {len(data)}, Feature length: {data.num_features}, Target length: {data.num_classes}")


Data length: 130831, Feature length: 11, Target length: 19


In [4]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool

from torch_geometric.datasets.qm9 import QM9

# Set the device
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print('MPS (multi-process service) is available')
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available')
else:
    device = torch.device('cpu')
    print('CUDA and MPS are not available, CPU is used')

# Define a simple Graph Neural Network
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x


# Split dataset (80% train, 10% val, 10% test)
train_dataset =data[:int(0.8 * len(data))]
val_dataset =data[int(0.8 * len(data)):int(0.9 * len(data))]
test_dataset =data[int(0.9 * len(data)):]


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True,num_workers=4, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True,num_workers=4, prefetch_factor=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True,num_workers=4, prefetch_factor=2)

model = GNN(input_dim=data.num_features, hidden_dim=64, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


MPS (multi-process service) is available


In [5]:
# Training loop
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y[:, 0].unsqueeze(1))  # Predicting the first target (mu)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Validation loop
def evaluate(loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            with torch.autocast(device='mps',dtype=torch.float16):
                data = data.to(device)
                output = model(data)
                loss = criterion(output, data.y[:, 0].unsqueeze(1))
                total_loss += loss.item()
    return total_loss / len(loader)

# Main training process
for epoch in range(50):
    train_loss = train()
    val_loss = evaluate(val_loader)
    print(f"Epoch {epoch + 1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Evaluate on the test set
test_loss = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}")

libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x11ad498a0>
Traceback (most recent call last):
  File "/Users/zeyudeng/miniconda3/envs/matsci/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/Users/zeyudeng/miniconda3/envs/matsci/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/zeyudeng/miniconda3/envs/matsci/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
        

KeyboardInterrupt: 